# ERDDAP

https://ioos.github.io/erddapy/

In [294]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
from ckanapi import RemoteCKAN
from erddapy import ERDDAP
import urllib
import os


In [295]:
ckan_url = 'https://cioosatlantic.ca/ckan/'
ckan = RemoteCKAN(ckan_url)  # User Agent options available if needed

In [297]:
results = ckan.action.package_search()

n = 0
package = results['results'][n]

print(f"Package {n+1} of {results['count']}:")
print()
print(f"package_id:\n {package['resources'][0]['package_id']}\n")

package = results['results'][n]
keys = ['title_translated', 'notes_translated', 
        'keywords', 'license_id', 
        'spatial', 'temporal-extent', 
        'state', 'progress']
for key in keys:

    if isinstance(package[key], dict) and 'en' in package[key]:
        value = package[key]['en']
    else:
        value = package[key]
    print(f"{key}:\n {value}\n")

print(f"url:\n {package['resources'][0]['url']}\n")

Package 1 of 34:

package_id:
 19258699-66b4-4c53-b0e4-09f2026e82dd

title_translated:
 NEGL, Cartwright Junction, NunatuKavut (NLQU0004)

notes_translated:
 NEGL - Cartwright Junction, NunatuKavut (NLQU0004). Tri-leg tower masts are 25 feet tall (7.62 m) and stations are all located at mid-slopes in cleared terrain.  Sites at Cartwright Junction, near Red Bay and at North West River all include fence enclosures which may impact snow accumulation at these sites.  All stations are owned and operated by the Northern Environmental Geoscience Laboratory led by Dr. Robert Way. All stations were established in partnership with the communities of Black Tickle, North West River, Postville, Red Bay and Rigolet.  Support for this project was received from the Labrador Institute of Memorial University of Newfoundland, the Nunatsiavut Government, the NunatuKavut Community Council, the Government of Newfoundland and Labrador's Department of Fisheries and Land Resources & Department of Municipal Aff

In [319]:
os.path.splitext('dfgg.dsafaghsdalfjdsf.html')

('dfgg.dsafaghsdalfjdsf', '.html')

In [320]:
def get_ERDDAP(id):
    ckan_url = 'https://cioosatlantic.ca/ckan/'
    ckan = RemoteCKAN(ckan_url)
    
    package = ckan.action.package_show(id=id)
    
    url = None
    for resource in package['resources']:
        if 'ERDDAP' in resource['name']:
            url = resource['url']
            break
            
    part1, part2 = url.split('erddap/')
    
    dataset_id, _ = os.path.splitext(os.path.basename(url))
    server = os.path.join(part1, 'erddap')
    protocol = part2.split('/')[0]
    
    e = ERDDAP(server=server, 
               protocol=protocol, response='nc')
    e.dataset_id = dataset_id
    
    return e

In [321]:
e = get_ERDDAP(package['resources'][0]['package_id'])
e

In [322]:
df = e.to_pandas()
df.head()

,Cruise,Event,Station,Type,time (UTC),longitude (degrees_east),latitude (degrees_north),BotDepth_m,Instrument,time_ISO8601,Pressure_dbar,Temperature_oC,Conductivity_S_m,Oxygen_V,Depth_m,Salinity (PSU),Oxygen_mL_L,Oxygen_mg_L,Oxygen,DescentRate_m_s
0,3872,1.0,NLSC214,C,2009-06-18T15:15:00Z,-56.316667,47.624667,60.0,SBE25#0466,NaN-NaN-NaNTNaN:NaN:NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,3872,1.0,NLSC214,C,2009-06-18T15:15:00Z,-56.316667,47.624667,60.0,SBE25#0466,NaN-NaN-NaNTNaN:NaN:NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,3872,1.0,NLSC214,C,2009-06-18T15:15:00Z,-56.316667,47.624667,60.0,SBE25#0466,2009-06-18T15:17:18,2.424250,9.306150,3.297322,3.449025,2.0,30.396850,6.732067,9.620800,101.941698,0.416500
3,3872,1.0,NLSC214,C,2009-06-18T15:15:00Z,-56.316667,47.624667,60.0,SBE25#0466,2009-06-18T15:17:19,3.004800,9.439450,3.295407,3.449320,3.0,30.264160,6.751930,9.649184,102.457585,0.691600
4,3872,1.0,NLSC214,C,2009-06-18T15:15:00Z,-56.316667,47.624667,60.0,SBE25#0466,2009-06-18T15:17:20,4.057533,9.251467,3.298328,3.455213,4.0,30.453207,6.791105,9.705166,102.748027,0.548867


In [286]:
ds = e.to_xarray()
ds

<xarray.Dataset>
Dimensions:           (row: 8860)
Coordinates:
    time              (row) datetime64[ns] ...
    latitude          (row) float64 ...
    longitude         (row) float64 ...
Dimensions without coordinates: row
Data variables:
    station_name      (row) object ...
    stat_num          (row) object ...
    wsc_num           (row) object ...
    air_temp_avg      (row) float32 ...
    rel_humidity      (row) float32 ...
    air_pressure_avg  (row) float32 ...
    precip_tb         (row) float32 ...
    rain              (row) object ...
    snow              (row) float32 ...
    snow_depth        (row) float64 ...
    wind_spd_avg      (row) float32 ...
    wind_dir_avg      (row) float32 ...
    wind_spd_max      (row) float32 ...
    wind_spd_max_dir  (row) float32 ...
    battery_voltage   (row) float32 ...
    air_temp_max      (row) float32 ...
    air_temp_min      (row) float32 ...
    precip_tot        (row) float32 ...
    rel_humidity_max  (row) float32 ...
    rel_humidity_min  (row) float32 ...
    pyr_solar         (row) float32 ...
    soil_moist        (row) float32 ...
    soil_cond         (row) float32 ...
    soil_temp         (row) float32 ...
Attributes:
    acknowledgement:           The community of Rigolet, Labrador Institute o...
    cdm_data_type:             Point
    contributor_name:          Government of Newfoundland and Labrador's Depa...
    contributor_role:          Initial publisher of data
    Conventions:               COARDS, CF-1.6, ACDD-1.3
    creator_email:             robert.way@queensu.ca
    creator_name:              NEGL at Queen's University
    creator_type:              institution
    creator_url:               https://www.labradorgeolab.ca/
    Easternmost_Easting:       -58.43936
    elevation:                 72 m above sea level
    featureType:               Point
    geospatial_lat_max:        54.17608
    geospatial_lat_min:        54.17608
    geospatial_lat_units:      degrees_north
    geospatial_lon_max:        -58.43936
    geospatial_lon_min:        -58.43936
    geospatial_lon_units:      degrees_east
    history:                   2020-08-11T16:48:27Z https://www.mae.gov.nl.ca...
    id:                        sma_negl_rigolet_nlqu0002
    infoUrl:                   http://www.smartatlantic.ca/station_alt.html?i...
    institution:               NEGL at Queen's University
    keywords:                  air, AIR_TEMP, AIR_TEMP_MAX, AIR_TEMP_MIN, ATM...
    keywords_fra:              other
    keywords_vocabulary:       GCMD Science Keywords
    license:                   The data may be used and redistributed for fre...
    Northernmost_Northing:     54.17608
    project:                   Northern Environmental Geoscience Laboratory (...
    publisher_email:           SmartBay@smartbay.ca
    publisher_name:            SmartAtlantic
    publisher_url:             https://www.smartatlantic.ca/
    sourceUrl:                 https://www.mae.gov.nl.ca/wrmd/ADRS/v6/Templat...
    Southernmost_Northing:     54.17608
    standard_name_vocabulary:  CF Standard Name Table v55
    summary:                   NEGL - Rigolet, Nunatsiavut (NLQU0002). Tri-le...
    summary_fra:               NEGL - Rigolet, Nunatsiavut (NLQU0002). Les mâ...
    testOutOfDate:             now-2hours
    time_coverage_end:         2020-08-11T15:52:00Z
    time_coverage_start:       2019-07-30T12:52:00Z
    title:                     NEGL, Rigolet, Nunatsiavut (NLQU0002)
    title_fra:                 NEGL, Rigolet, Nunatsiavut (NLQU0002)
    uuid:                      147423a6-b150-4cf5-9338-d9cbfb7452db
    Westernmost_Easting:       -58.43936

In [350]:
packages = ckan.action.package_list()

for id in packages:
    if id=='cioos-atlantic-waf':
        # Ignore CIOOS Atlantic WAF
        continue
        
    if id=='ca-meopar_bb1_bop_level1':
        continue
        
    e = get_ERDDAP(id)
    print(id)
    print(e.get_download_url())
    
    ncfile = os.path.join("data", e.dataset_id + '.nc')
    
    if not os.path.exists(ncfile):
        ds = e.to_xarray()
        ds.to_netcdf(ncfile)

0c789db6-ae46-4899-9d7e-eb151bf49fdd
https://www.smartatlantic.ca/erddap/tabledap/DFO_Sutron_NHARB.nc?
0fb1bc5e-0eb6-4dba-b494-0b80845b2030
https://www.smartatlantic.ca/erddap/tabledap/SMA_halifax.nc?
147423a6-b150-4cf5-9338-d9cbfb7452db
https://www.smartatlantic.ca/erddap/tabledap/sma_negl_rigolet_nlqu0002.nc?
2ed26c3c-d328-453d-86cb-eef7ab3f9c58
https://www.smartatlantic.ca/erddap/tabledap/SMA_manolis_buoy.nc?
354ef3d6-4d39-48e4-8adc-518fbb425752
https://www.smartatlantic.ca/erddap/tabledap/SMA_port_aux_basques.nc?
38882a03-33a7-45b9-b2bd-916268f1e073
https://www.smartatlantic.ca/erddap/tabledap/SMA_head_of_placentia_bay-come_by_chance_point.nc?
4bf9f57d-f7c1-4522-b7ec-82deafb0bb31
https://www.smartatlantic.ca/erddap/tabledap/DFO_Sutron_KLUMI.nc?
61f4cc32-0833-406f-80da-b725418239f3
https://www.smartatlantic.ca/erddap/tabledap/SMA_saint_john_cruise_terminal.nc?
65931687-684a-41b5-9aa5-086a9e72b5c5
https://www.smartatlantic.ca/erddap/tabledap/sma_negl_postville_nlqu0001.nc?
69423be3-f

In [310]:
package = ckan.action.package_show(id=id)

In [315]:
package['resources'][0]

{'cache_last_updated': None,
 'package_id': '4f5ab492-0ba4-442e-9ec1-2093b64b20f4',
 'datastore_active': False,
 'id': 'c9c3ed24-a4a0-4211-b617-2cb4d5f8c1cf',
 'size': None,
 'state': 'active',
 'resource_locator_function': 'download',
 'hash': '',
 'description': "ERDDAP's version of the OPeNDAP .html web page for this dataset. Specify a subset of the dataset and download the data via OPeNDAP or in many different file types.",
 'format': 'HTML',
 'mimetype_inner': None,
 'url_type': None,
 'resource_locator_protocol': 'order',
 'mimetype': None,
 'cache_url': None,
 'name': 'ERDDAP Data Subset Form',
 'created': '2020-06-12T12:36:22.562618',
 'url': 'https://cioosatlantic.ca/erddap/tabledap/coast-of-bays-hydrographic-2009-2013.html',
 'last_modified': None,
 'position': 0,
 'revision_id': '1a6cb452-20e0-4934-b345-003b6333464d',
 'resource_type': None}